In [33]:
#Set-up
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
from IPython.display import clear_output
import nibabel as nib

import torch
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.utils.data as utils
import time
import copy

In [34]:
x = torch.load("data.pt")
y = torch.load("labels.pt")
n = list(x.size())[0]
print(n)

536


In [35]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [36]:
dataset = utils.TensorDataset(x, y)

In [37]:
dataloaders = {i: torch.utils.data.DataLoader(dataset, batch_size=4,
                                             shuffle=True, num_workers=4)
              for i in ['train', 'val']}

In [38]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_offset = float('inf')

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_offset = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_offset += torch.sum(torch.abs(outputs - labels.data))

            epoch_loss = running_loss / n
            epoch_offset = running_offset / n

            print('{} Loss: {:.4f} Diff: {:.4f}'.format(
                phase, epoch_loss, epoch_offset))

            # deep copy the model
            if phase == 'val' and epoch_offset < best_offset:
                best_offset = epoch_offset
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Diff: {:4f}'.format(best_offset))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [39]:
model_ft = models.resnet18(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 1)

model_ft = model_ft.to(device)

criterion = nn.MSELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [40]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=50)

Epoch 0/49
----------
train Loss: 527.8100 Diff: 18.3018
val Loss: 550.6073 Diff: 19.7334

Epoch 1/49
----------
train Loss: 285.7726 Diff: 14.1168
val Loss: 377.6499 Diff: 15.8906

Epoch 2/49
----------
train Loss: 254.3019 Diff: 13.1638
val Loss: 210.4305 Diff: 12.2545

Epoch 3/49
----------
train Loss: 258.0933 Diff: 13.3870
val Loss: 178.9872 Diff: 11.2418

Epoch 4/49
----------
train Loss: 247.1943 Diff: 12.6899
val Loss: 1384.8518 Diff: 34.7980

Epoch 5/49
----------
train Loss: 241.2976 Diff: 12.6866
val Loss: 161.0139 Diff: 9.9382

Epoch 6/49
----------
train Loss: 174.1513 Diff: 10.5533
val Loss: 142.6445 Diff: 9.4961

Epoch 7/49
----------
train Loss: 170.2500 Diff: 10.4711
val Loss: 140.7156 Diff: 9.5377

Epoch 8/49
----------
train Loss: 178.7556 Diff: 10.7732
val Loss: 136.9044 Diff: 9.5109

Epoch 9/49
----------
train Loss: 175.1633 Diff: 10.6676
val Loss: 139.1009 Diff: 9.6223

Epoch 10/49
----------
train Loss: 173.9723 Diff: 10.7610
val Loss: 132.8174 Diff: 9.2555

Epo